In [7]:
import pandas as pd
import time
import os

# import data
file = "/Users/hongling/Desktop/association_rules_Apriori_algorithm/Sample - Superstore.xls"
data = pd.read_excel(file, sheet_name = "Orders")

t0=time.time()

#select columns needed
data1 = data[["Order ID", "Product ID"]] 
#calculate the times each product showed up (i.e. n_X)
data21 = data.groupby('Product ID').size().reset_index()
#rename columns
data21.columns = ["XID", "n_X"] 
#calculate the times each product showed up (i.e. n_Y)
data22 = data.groupby('Product ID').size().reset_index() 
#rename columns
data22.columns = ["YID", "n_Y"]
#outer join two tables containing OID and PID, based on Order ID
data3 = pd.merge(data1, data1, how = "outer", on = ["Order ID"]) 
#rename columns
data3.columns = ["Order ID", "XID", "YID"] 
#delete repete ones where X and Y are the same
data3 = data3.query("XID != YID") 
#calculate the times when X and Y showed up together (n_XY)
data4 = data3.groupby(['XID','YID']).size().reset_index()
#rename columns
data4.columns = ["XID", "YID", "n_XY"] 
#merge n_XY and n_X in one table
data5 = pd.merge(data4, data21, on = "XID")
#add n_Y to the previous table
data5 = pd.merge(data5, data22, on = "YID")
#add column: calculate x_Transactions
data5["n_Transactions"] = len(set(data1["Order ID"]))
#add column: calculate Support(X)=n(X)/n(Total Transactions)
data5["Support"] = data5["n_X"]/data5["n_Transactions"]
#add column: calculate Confidence（X->Y)=n(XY)/n(X)
data5["Confidence"] = data5["n_XY"]/data5["n_X"] 
#add column: calculate Lift(X->Y)=Confidence(X->Y)/n(Y)
data5["Lift"] = data5["Confidence"]/data5["n_Y"]
#final version
data5

print("total execution time:", time.time()-t0)

total execution time: 0.044523000717163086


In [4]:
data5

,XID,YID,n_XY,n_X,n_Y,n_Transactions,Support,Confidence,Lift
0,FUR-BO-10000112,OFF-AR-10001149,1,1,7,5009,0.000200,1.000000,0.142857
1,FUR-CH-10004289,OFF-AR-10001149,1,7,7,5009,0.001397,0.142857,0.020408
2,FUR-CH-10004997,OFF-AR-10001149,1,10,7,5009,0.001996,0.100000,0.014286
3,FUR-FU-10001706,OFF-AR-10001149,1,10,7,5009,0.001996,0.100000,0.014286
4,FUR-TA-10003569,OFF-AR-10001149,1,6,7,5009,0.001198,0.166667,0.023810
...,...,...,...,...,...,...,...,...,...
19781,TEC-PH-10001198,OFF-PA-10001001,1,4,1,5009,0.000799,0.250000,0.250000
19782,TEC-PH-10001254,OFF-AP-10000326,1,4,1,5009,0.000799,0.250000,0.250000
19783,TEC-PH-10001817,TEC-MA-10001031,1,10,1,5009,0.001996,0.100000,0.100000
19784,TEC-PH-10002726,OFF-AR-10002671,1,7,1,5009,0.001397,0.142857,0.142857


In [5]:
with pd.ExcelWriter('Sample Superstore Association Results.xls') as writer:
    data5.to_excel(writer,sheet_name="Summary")